In [ ]:
import math
import io

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

matplotlib.rcParams['figure.dpi'] = 100

# Diagnostics

Use `./run.sh` in the root directory to dump diagnostic variables to `simple-toplogy-stderr.txt`. Then use this notebook for plots


## Packet Ack Diagnostics

(stuff we collect on every ack). 

plots are shaded green when the algorithm is learning

In [ ]:
# Get data for every ACK
ack_data = "".join(i for i in open("../simple-topology-stderr.txt") if "ack" in i)
df_ack = pd.read_csv(io.StringIO(ack_data))
df_ack = df_ack.drop("ack", axis=1)
df_ack.columns = ["time", "variable", "value"]
df_ack = df_ack.pivot(index="time", columns="variable", values="value")
df_ack = df_ack.reindex(sorted(df_ack.columns), axis=1)

# Get data for every execution of Learn()
learn_data = "".join(i for i in open("../simple-topology-stderr.txt") if "learn" in i)
df_learn = pd.read_csv(io.StringIO(learn_data))
df_learn = df_learn.drop("learn", axis=1)
df_learn.columns = ["time", "variable", "value"]
df_learn = df_learn.pivot(index="time", columns="variable", values="value")
df_learn = df_learn.reindex(sorted(df_learn.columns), axis=1)

In [ ]:
df_ack.describe()

In [ ]:
df_learn.describe()

In [ ]:
def plot_learn_events(ax):
    ax.vlines(df_learn.index, *ax.get_ylim(), alpha=0.1, color="green", linewidth=0.5)

In [ ]:
state_cols = [i for i in df_ack.columns if "state" in i]
fig, ax = plt.subplots()
for i, col in enumerate(state_cols):
    ax.plot(df_ack.index, df_ack[col], label=i)
plot_learn_events(ax)
ax.legend()
ax.set_title("State Variables")

In [ ]:
cols_to_plot = [i for i in df_ack.columns if "state" not in i]
n_rows = math.ceil(len(cols_to_plot) / 2)
fig, axs = plt.subplots(n_rows, 2, figsize=(6, 2 * n_rows))

for i, col in enumerate(cols_to_plot):
    ax = axs.flatten()[i]
    df_ack.plot(y=col, use_index=True, ax=ax, legend=None)
    plot_learn_events(ax)
    ax.set_title(col)
fig.tight_layout()

## Learn diagnostics

(Stuff we collect every time the FuzzyKanerva function is executed)

In [ ]:
cols_to_plot = [i for i in df_learn.columns if "state" not in i]
n_rows = math.ceil(len(cols_to_plot) / 2)
fig, axs = plt.subplots(n_rows, 2, figsize=(6, 2 * n_rows))

for i, col in enumerate(cols_to_plot):
    ax = axs.flatten()[i]
    df_learn.plot(y=col, use_index=True, ax=ax, legend=None)
    plot_learn_events(ax)
    ax.set_title(col)
fig.tight_layout()